In [2]:
from google.cloud import bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials/endless-beach-228506-b15de187c224.json"

In [3]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "world_bank_intl_education" dataset
dataset_ref = client.dataset("world_bank_intl_education", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "international_education" table
table_ref = dataset_ref.table("international_education")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "international_education" table
client.list_rows(table, max_results=20).to_dataframe()

,country_name,country_code,indicator_name,indicator_code,value,year
0,Marshall Islands,MHL,Population of the official entrance age to sec...,UIS.SAP.23.GPV.G1.F,6.810000e+02,2016
1,Mauritania,MRT,"Enrolment in tertiary education, all programme...",SE.TER.ENRL,2.028000e+04,2016
2,Mauritania,MRT,Population of the official entrance age to sec...,UIS.SAP.23.GPV.G1.M,4.913800e+04,2016
3,Mauritania,MRT,"Population, ages 0-14, female",SP.POP.0014.FE.IN,8.483770e+05,2016
4,"Micronesia, Fed. Sts.",FSM,"GDP, PPP (current international $)",NY.GDP.MKTP.PP.CD,3.689088e+08,2016
5,St. Lucia,LCA,Population of the official age for post-second...,UIS.SAP.4.F,3.211000e+03,2016
6,St. Lucia,LCA,Theoretical duration of pre-primary education ...,UIS.THDUR.0,2.000000e+00,2016
7,Suriname,SUR,Official entrance age to upper secondary educa...,UIS.THAGE.3.A.GPV,1.600000e+01,2016
8,Tajikistan,TJK,"Gross enrolment ratio, pre-primary, male (%)",SE.PRE.ENRR.MA,1.175560e+01,2016
9,Tajikistan,TJK,Percentage of male students enrolled in primar...,UIS.OAEP.1.M,1.159600e-01,2016


In [15]:
# Your code goes here
country_spend_pct_query = """
                          SELECT country_name, AVG(value) as avg_ed_spending_pct
                          FROM `bigquery-public-data.world_bank_intl_education.international_education`
                          WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' AND year >= 2010 AND year <=2017
                          GROUP BY country_name
                          ORDER BY avg_ed_spending_pct DESC
                          """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig()
country_spend_pct_query_job = client.query(country_spend_pct_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
country_spending_results = country_spend_pct_query_job.to_dataframe()

# View top few rows of results
print(country_spending_results.head())

            country_name  avg_ed_spending_pct
0                   Cuba            12.837270
1  Micronesia, Fed. Sts.            12.467750
2        Solomon Islands            10.001080
3                Moldova             8.372153
4                Namibia             8.349610


In [23]:
# Your code goes here
country_spend_pct_query = """
                          SELECT indicator_code, indicator_name, COUNT(1) AS num_rows
                          FROM `bigquery-public-data.world_bank_intl_education.international_education`
                          WHERE year = 2016
                          GROUP BY indicator_name, indicator_code
                          HAVING num_rows >= 175
                          ORDER BY num_rows DESC
                          """
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig()
country_spend_pct_query_job = client.query(country_spend_pct_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
country_spending_results = country_spend_pct_query_job.to_dataframe()

# View top few rows of results
print(country_spending_results.head())

      indicator_code                       indicator_name  num_rows
0        SP.POP.TOTL                    Population, total       232
1        SP.POP.GROW         Population growth (annual %)       232
2     IT.NET.USER.P2      Internet users (per 100 people)       223
3  SP.POP.1564.TO.ZS  Population, ages 15-64 (% of total)       213
4  SP.POP.TOTL.FE.ZS      Population, female (% of total)       213
